In [1]:
import time
import numpy as np
import random
import cv2
import numpy as np
import RPi.GPIO as GPIO
from time import sleep
from collections import Counter
import threading

in1 = 24
in2 = 23
ena = 25
in3 = 17
in4 = 27
enb = 18
temp1=1

GPIO.setmode(GPIO.BCM)
GPIO.setup(in1,GPIO.OUT)
GPIO.setup(in2,GPIO.OUT)
GPIO.setup(ena,GPIO.OUT)
GPIO.setup(in3,GPIO.OUT)
GPIO.setup(in4,GPIO.OUT)
GPIO.setup(enb,GPIO.OUT)

# Set Low
GPIO.output(in1,GPIO.LOW)
GPIO.output(in2,GPIO.LOW)
GPIO.output(in3,GPIO.LOW)
GPIO.output(in4,GPIO.LOW)

# PWM with 100 duty cycle
q=GPIO.PWM(enb,10)
p=GPIO.PWM(ena,10)
p.start(100)
q.start(100)


# Time (in seconds) to Run the action
# Fine tune it according to your rover
t_forward = 0.61
t_turn = 0.295


# Handling individual pins
def start_pin(pin, time):
    GPIO.output(pin, GPIO.HIGH)
    t = threading.Timer(time, end)
    t.start()

def end():
    GPIO.output(in1,GPIO.LOW)
    GPIO.output(in2,GPIO.LOW)
    GPIO.output(in3,GPIO.LOW)
    GPIO.output(in4,GPIO.LOW)


# Call these functions to execute unit movements

def move_forword():
    start_pin(in1, t_forward)
    start_pin(in3, t_forward)
    sleep(t_forward)

def turn_left():
    start_pin(in3, t_turn)
    start_pin(in2, t_turn)
    sleep(t_turn)
    
def turn_right():
    start_pin(in1, t_turn)
    start_pin(in4, t_turn)
    sleep(t_turn)


# Calculate Mode
def calculate_mode(numbers):
    counter = Counter(numbers)
    mode = counter.most_common(1)[0][0]
    return mode

def Detect_color():
    cap=cv2.VideoCapture(0)
    
    # Proportion of the colour
    percent_pink  = percent_green = percent_yellow = percent_sky_blue = percent_purple = percent_black= 0
    colour_index = [] # just for internal computation

    loop_count = 10
    while(loop_count>0):
        # Reading the video from the
        # webcam in image frames
        _, imageFrame = cap.read()
        #imageFrame=cv2.resize(imageFrame,(300,300))

        # Convert the imageFrame in
        # BGR(RGB color space) to
        # HSV(hue-saturation-value)
        # color space
        hsvFrame = cv2.cvtColor(imageFrame, cv2.COLOR_BGR2HSV)


        # Colour Filters
        # The test setup will have the following color mappings
        # Purple - Wumpus
	# Pink - Stench
	# Blue - Breeze
	# Green - Pit
	# Yellow - Gold
	#
	# The following values have been calibrated for the test setup in 105
	#
        pink_lower = np.array([148,43,34], np.uint8)
        pink_upper = np.array([179,255,255], np.uint8)
        pink_mask = cv2.inRange(hsvFrame, pink_lower, pink_upper)
        pink_count = np.sum(np.nonzero(pink_mask))

        #cv2.imshow("pink",pink_mask)

        # Set range for green color and
        # define mask
        green_lower = np.array([31,40,45], np.uint8)
        green_upper = np.array([80,255,190], np.uint8)
        green_mask = cv2.inRange(hsvFrame, green_lower, green_upper)
        green_count = np.sum(np.nonzero(green_mask))

        # Set range for yellow color and
        # define mask
        yellow_lower = np.array([0,75,50], np.uint8)
        yellow_upper = np.array([44,255,168], np.uint8)
        yellow_mask = cv2.inRange(hsvFrame, yellow_lower, yellow_upper)
        yellow_count = np.sum(np.nonzero(yellow_mask))

        # Set range for sky_blue color and
        # define mask
        sky_blue_lower = np.array([70,40,120], np.uint8)
        sky_blue_upper = np.array([137,255,255], np.uint8)
        sky_blue_mask = cv2.inRange(hsvFrame, sky_blue_lower, sky_blue_upper)
        sky_blue_count = np.sum(np.nonzero(sky_blue_mask))

        # Set range for purple color and
        # define mask
        purple_lower = np.array([68,10,2], np.uint8)
        purple_upper = np.array([152,154,229], np.uint8)
        purple_mask = cv2.inRange(hsvFrame, purple_lower, purple_upper)
        purple_count = np.sum(np.nonzero(purple_mask))

        # Set range for black color and
        # define mask
        black_lower = np.array([23,143,0], np.uint8)
        black_upper = np.array([179,255,40], np.uint8)
        black_mask = cv2.inRange(hsvFrame, black_lower, black_upper)
        black_count = np.sum(np.nonzero(black_mask))

        total_count = pink_count + green_count + yellow_count + sky_blue_count + purple_count + black_count
        
        # Individual Share
        percent_pink  = pink_count*100/total_count
        percent_green = green_count*100/total_count
        percent_yellow = yellow_count*100/total_count
        percent_sky_blue = sky_blue_count*100/total_count
        percent_purple = purple_count*100/total_count
        percent_black= black_count*100/total_count
    
        # each loop has a max detected colour
        frame_list = [percent_pink,percent_green,percent_yellow,percent_sky_blue,percent_purple,percent_black]
        colour_index.append(frame_list.index(max(frame_list)))

        loop_count-=1

    colour = calculate_mode(colour_index)
    print(colour)


    cap.release()
    cv2.destroyAllWindows()

    if colour == 0:
        print("stench: pink")
        return "pink"
    elif colour == 1:
        print("pit: green")
        return "green"
    elif colour == 2:
        print("gold: yellow")
        return "yellow"
    elif colour == 3:
        print("sky_blue")
        return "breeze: sky_blue"
    elif colour == 4:
        print("wumpus: purple")
        return "purple"
    elif colour == 5:
        print("stop: black")
        return "black"
    

class Sentence():

    def evaluate(self, model):
        """Evaluates the logical sentence."""
        raise Exception("nothing to evaluate")

    def formula(self):
        """Returns string formula representing logical sentence."""
        return ""

    def symbols(self):
        """Returns a set of all symbols in the logical sentence."""
        return set()

    @classmethod
    def validate(cls, sentence):
        if not isinstance(sentence, Sentence):
            raise TypeError("must be a logical sentence")

    @classmethod
    def parenthesize(cls, s):
        """Parenthesizes an expression if not already parenthesized."""
        def balanced(s):
            """Checks if a string has balanced parentheses."""
            count = 0
            for c in s:
                if c == "(":
                    count += 1
                elif c == ")":
                    if count <= 0:
                        return False
                    count -= 1
            return count == 0
        if not len(s) or s.isalpha() or (
            s[0] == "(" and s[-1] == ")" and balanced(s[1:-1])
        ):
            return s
        else:
            return f"({s})"


class Symbol():

    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return isinstance(other, Symbol) and self.name == other.name

    def __hash__(self):
        return hash(("symbol", self.name))

    def __repr__(self):
        return self.name

    def evaluate(self, model):
        try:
            return bool(model[self.name])
        except KeyError:
            raise EvaluationException(f"variable {self.name} not in model")

    def formula(self):
        return self.name

    def symbols(self):
        return {self.name}


class Not():
    def __init__(self, operand):
        #Sentence.validate(operand)
        self.operand = operand

    def __eq__(self, other):
        return isinstance(other, Not) and self.operand == other.operand

    def __hash__(self):
        return hash(("not", hash(self.operand)))

    def __repr__(self):
        return f"Not({self.operand})"

    def evaluate(self, model):
        return not self.operand.evaluate(model)

    def formula(self):
        return "Â¬" + Sentence.parenthesize(self.operand.formula())

    def symbols(self):
        return self.operand.symbols()


class And():
    def __init__(self, *conjuncts):
        #for conjunct in conjuncts:
           # Sentence.validate(conjunct)
        self.conjuncts = list(conjuncts)

    def __eq__(self, other):
        return isinstance(other, And) and self.conjuncts == other.conjuncts

    def __hash__(self):
        return hash(
            ("and", tuple(hash(conjunct) for conjunct in self.conjuncts))
        )

    def __repr__(self):
        conjunctions = ", ".join(
            [str(conjunct) for conjunct in self.conjuncts]
        )
        return f"And({conjunctions})"

    def add(self, conjunct):
        #Sentence.validate(conjunct)
        self.conjuncts.append(conjunct)

    def evaluate(self, model):
        return all(conjunct.evaluate(model) for conjunct in self.conjuncts)

    def formula(self):
        if len(self.conjuncts) == 1:
            return self.conjuncts[0].formula()
        return " âˆ§ ".join([Sentence.parenthesize(conjunct.formula())
                           for conjunct in self.conjuncts])

    def symbols(self):
        return set.union(*[conjunct.symbols() for conjunct in self.conjuncts])


class Or():
    def __init__(self, *disjuncts):
        #for disjunct in disjuncts:
           # Sentence.validate(disjunct)
        self.disjuncts = list(disjuncts)

    def __eq__(self, other):
        return isinstance(other, Or) and self.disjuncts == other.disjuncts

    def __hash__(self):
        return hash(
            ("or", tuple(hash(disjunct) for disjunct in self.disjuncts))
        )

    def __repr__(self):
        disjuncts = ", ".join([str(disjunct) for disjunct in self.disjuncts])
        return f"Or({disjuncts})"

    def evaluate(self, model):
        return any(disjunct.evaluate(model) for disjunct in self.disjuncts)

    def formula(self):
        if len(self.disjuncts) == 1:
            return self.disjuncts[0].formula()
        return " âˆ¨  ".join([Sentence.parenthesize(disjunct.formula())
                            for disjunct in self.disjuncts])

    def symbols(self):
        return set.union(*[disjunct.symbols() for disjunct in self.disjuncts])


class Implication():
    def __init__(self, antecedent, consequent):
        #Sentence.validate(antecedent)
        #Sentence.validate(consequent)
        self.antecedent = antecedent
        self.consequent = consequent

    def __eq__(self, other):
        return (isinstance(other, Implication)
                and self.antecedent == other.antecedent
                and self.consequent == other.consequent)

    def __hash__(self):
        return hash(("implies", hash(self.antecedent), hash(self.consequent)))

    def __repr__(self):
        return f"Implication({self.antecedent}, {self.consequent})"

    def evaluate(self, model):
        return ((not self.antecedent.evaluate(model))
                or self.consequent.evaluate(model))

    def formula(self):
        antecedent = Sentence.parenthesize(self.antecedent.formula())
        consequent = Sentence.parenthesize(self.consequent.formula())
        return f"{antecedent} => {consequent}"

    def symbols(self):
        return set.union(self.antecedent.symbols(), self.consequent.symbols())


class Biconditional():
    def __init__(self, left, right):
       # Sentence.validate(left)
       # Sentence.validate(right)
        self.left = left
        self.right = right

    def __eq__(self, other):
        return (isinstance(other, Biconditional)
                and self.left == other.left
                and self.right == other.right)

    def __hash__(self):
        return hash(("biconditional", hash(self.left), hash(self.right)))

    def __repr__(self):
        return f"Biconditional({self.left}, {self.right})"

    def evaluate(self, model):
        return ((self.left.evaluate(model)
                 and self.right.evaluate(model))
                or (not self.left.evaluate(model)
                    and not self.right.evaluate(model)))

    def formula(self):
        left = Sentence.parenthesize(str(self.left))
        right = Sentence.parenthesize(str(self.right))
        return f"{left} <=> {right}"

    def symbols(self):
        return set.union(self.left.symbols(), self.right.symbols())


def model_check(knowledge,pre_symbols=set(),model=[dict()]):#, query,model=dict()
    KBB = []
    ALL_model = []
    def check_all(knowledge, symbols, model): #query,
        if not symbols:
            if knowledge.evaluate(model):
                #KB
                KBB.append(model)
                #print("ll")
                #print(model)
                #return query.evaluate(model)
                return True
                
            #else:
                #print("lol lol:",model)
            return True
        else:
            remaining = symbols.copy()
            p = remaining.pop()
            model_true = model.copy()
            model_true[p] = True
            model_false = model.copy()
            model_false[p] = False

            # Ensure entailment holds in both models
            #return (check_all(knowledge, query, remaining, model_true) and
                    #check_all(knowledge, query, remaining, model_false)
                
            return (check_all(knowledge, remaining, model_true) and
                    check_all(knowledge, remaining, model_false))

    # Get all symbols in both knowledge and query
    #symbols = set.union(knowledge.symbols(), query.symbols())
    #
    #print(symbols)
    s_symbols = knowledge.symbols()
    symbols = s_symbols.difference(pre_symbols)
    # Check that knowledge entails query
    #return check_all(knowledge, query, symbols, model)
    #return check_all(knowledge, symbols, model),KBB
    for mini_mod in model:
        #print(mini_mod)
        check_all(knowledge, symbols, mini_mod)
        ALL_model.append(KBB)
        KBB = []
    for i in ALL_model:
        for j in i:
            KBB.append(j)
            
    return KBB,s_symbols

def check_res(KB_models,query,check):
    for ji in KB_models:
        if(ji[query]!=check):
            return False
    return True
    
class make_agent():
    
    def __init__(self,n):
        self.start = [n-1,0]
        self.visited = np.zeros((n,n))
        self.currant = self.start
        self.curr_di = 0
        self.move_di = {0:[[0,1],'-> '],
                       1:[[-1,0],'|^ '],
                       2:[[0,-1],'<- '],
                       3:[[1,0],'_| ']}
        self.size = n
        
    def move_forward(self):
        if((self.curr_di==0 and self.currant[1]==self.size-1) or (self.curr_di==1 and self.currant[0]==0) or (self.curr_di==2 and self.currant[1]==0) or (self.curr_di==3 and self.currant[0]==self.size-1)):
            pass
        else:
            self.currant = [sum(i) for i in zip(self.currant, self.move_di[self.curr_di][0])]
        #55#move_forword()
        
        ## car move
        
    def turn_90(self,pan):
        if(self.curr_di == 0 and pan == -1):
            pan = 3
        if(self.curr_di == 3 and pan == 1):
            pan = -3
        self.curr_di = self.curr_di+pan
        
        
        ## move car
        
    def make_turn(self,dirr):
        if(dirr=='right'):
            self.turn_90(-1)
            #55#turn_right()
        if(dirr=='left'):
            self.turn_90(1)
            #55#turn_left()
        
        
class block():
    
    def __init__(self):
        self.gold = False
        self.pit = False
        self.wumpus = False
        self.breeze = False
        self.stench = False
        self.adj = []
        self.neighbor = []
        self.posi = []
        self.visited = False
        self.ok = False
        self.children = []
        self.parent= None
        
    def add_child(self,child):
        child.parent = self
        self.children.append(child)
        
        
class make_wumpus_world:
    def __init__(self,n):
        
        self.size = n
        self.blocks = [block() for i in range(n**2)]
        random_sample = random.sample(self.blocks[1:],2+int(0.2 * (n**2)))
        self.start = [self.blocks[0]]
        #self.gold = [random_sample[0]]
        #self.wumpus = [random_sample[1]]
        #self.pits = random_sample[2:]
        self.set_grid()
        #self.grid = np.flipud(self.grid)
        self.agent = make_agent(self.size)
        self.kb = And()
        self.dic = {True:'',False:'Not'}
        self.win = False
   
    def set_grid(self):
        
        self.grid = np.asarray(self.blocks).reshape(self.size,self.size)
        self.grid = np.flipud(self.grid)
        self.get_nie()
        #self.gold[0].gold = True
        #self.wumpus[0].wumpus = True
        #for i in self.wumpus[0].neighbor:
            #i.stench = True
        #for i in self.pits:
            #i.pit = True
            #for j in i.neighbor:
                #j.breeze = True
                
    def start_state(self):
        self.di = {}
        lists = ['P','B','G','W','S']
        for i in range(self.size):
            for j in range(self.size):
                for k in lists:
                    self.di[f'{k}{i}{j}'] = Symbol(f'{k}{i}{j}')
        
        
    def get_nie(self):
        self.rule_neighbor = [[[] for i in range(self.size)] for j in range(self.size)]
        for i in range(self.size):
            for j in range(self.size):
                self.grid[i,j].posi = [i,j]
                if(i!=0):
                    self.grid[i,j].neighbor.append(self.grid[i-1,j])
                    self.rule_neighbor[i][j].append(f'{i-1}{j}')
                if(j!=self.size-1):
                    self.grid[i,j].neighbor.append(self.grid[i,j+1])
                    self.rule_neighbor[i][j].append(f'{i}{j+1}')
                if(i!=self.size-1):
                    self.grid[i,j].neighbor.append(self.grid[i+1,j])
                    self.rule_neighbor[i][j].append(f'{i+1}{j}')
                if(j!=0):
                    self.grid[i,j].neighbor.append(self.grid[i,j-1])
                    self.rule_neighbor[i][j].append(f'{i}{j-1}')

    def join_sym(self,smb,terms,extra='',extra2=''):
        temp = f'{smb}('
        for i in terms:
            temp = temp +f' {extra}{i}{extra2},'
        temp = temp[:-1] + ')'
        return temp
                    
    def rule_add(self):
        i = self.agent.currant[0]
        j = self.agent.currant[1]
        self.kb.add(eval(self.join_sym("Biconditional",[f"self.di['B{i}{j}']",self.join_sym('Or',self.rule_neighbor[i][j],"self.di['P","']")])))
        self.kb.add(eval(self.join_sym("Biconditional",[f"self.di['S{i}{j}']",self.join_sym('Or',self.rule_neighbor[i][j],"self.di['W","']")])))
        for ii in range(self.size):
            for jj in range(self.size):
                if self.agent.visited[ii,jj]==1:
                    self.kb.add(eval(self.join_sym("Or",[self.join_sym('Not',[f"self.di['G{i}{j}']"]),self.join_sym('Not',[f"self.di['G{ii}{jj}']"])])))
                    self.kb.add(eval(self.join_sym("Or",[self.join_sym('Not',[f"self.di['W{i}{j}']"]),self.join_sym('Not',[f"self.di['W{ii}{jj}']"])])))
        
        
        #print(self.join_sym(self.dic[self.grid[self.agent.currant[0],self.agent.currant[1]].gold],[f"self.di['G{ii}{jj}']"]))
        self.kb.add(eval(self.join_sym(self.dic[self.grid[self.agent.currant[0],self.agent.currant[1]].gold],[f"self.di['G{i}{j}']"])))
        #self.kb.add(eval(self.join_sym(self.dic[self.grid[self.agent.currant[0],self.agent.currant[1]].pit],[f"self.di['P{i}{j}']"])))
        #self.kb.add(eval(self.join_sym(self.dic[self.grid[self.agent.currant[0],self.agent.currant[1]].wumpus],[f"self.di['W{i}{j}']"])))
        self.kb.add(eval(self.join_sym(self.dic[self.grid[self.agent.currant[0],self.agent.currant[1]].breeze],[f"self.di['B{i}{j}']"])))
        self.kb.add(eval(self.join_sym(self.dic[self.grid[self.agent.currant[0],self.agent.currant[1]].stench],[f"self.di['S{i}{j}']"])))
        #print(self.kb)
        self.agent.visited[i,j]=1
                    
    def check_gold(self):
        if(self.grid[self.agent.currant[0],self.agent.currant[1]].gold==True):
            return True
        return False
    
    def check_wumpus(self):
        if(self.grid[self.agent.currant[0],self.agent.currant[1]].wumpus==True):
            return True
        return False
            
    def check_pit(self):
        if(self.grid[self.agent.currant[0],self.agent.currant[1]].pit==True):
            return True
        return False
    
    
    def get_array(self,pre=None):
        if pre != None:
            return [0,0,0,0]
        """write code hear"""
        #col = Detect_color()
        col = input()
        if(col=='yellow'):
            self.grid[self.agent.currant[0],self.agent.currant[1]].gold = True
            return [1,0,0,0,0]
        elif(col=='green'):
            self.grid[self.agent.currant[0],self.agent.currant[1]].breeze = True
            return [0,0,0,1,0]
        elif(col=='pink'):
            self.grid[self.agent.currant[0],self.agent.currant[1]].stench = True
            return [0,0,0,0,1]
        elif(col=="black"):
            return [0,0,0,0,0]
    
    def print_block(self,block,k,l):
        
        #if ()
        
        string = ''
        
        
        if(self.agent.currant==[k,l]):
            string += ' Agent '
            
            string += self.agent.move_di[self.agent.curr_di][1]
                #string += '-> '

            
        if(block.gold==True):
            string += ' gold '
            
        if(block.pit==True):
            string += ' pit '
        if(block.wumpus==True):
            string += ' wumpus '
        if(block.breeze==True):
            string += ' breeze '
        if(block.stench==True):
            string += ' stench '
        return string
    
    def print_grid(self):
        for i in range(self.size):
            for j in range(self.size):
                print(np.char.center(self.print_block(self.grid[i,j],i,j),25),end='')
            print()
        
class Game:
    def __init__(self):

        self.wumpus_world = make_wumpus_world(4)
        self.wumpus_world.start_state()
        self.listed_do_p = []
        self.listed_do_w = []
        self.listed_done_p = []
        self.listed_done_w = []
        self.list_done_wumpus = []
        self.listed_w = []
        self.Total_sym = {}
        self.predi_models = [{}]
        self.ok = []
        self.moves = []
        self.back = {"F":"Back",
                    "L":"R",
                    "R":"L"}
        self.wumpus_world.win = False
        self.cont = 0
        self.ch_f = False
        self.priv = [3,0]
        self.track = [[3,0]]
        

    def start(self):
        print(self.wumpus_world.agent.currant)
        #self.wumpus_world.get_array(45)
        self.wumpus_world.rule_add()
        self.evu()
        #print("ok its",self.ok)
        self.temp_move()
        while 1:
            #sleep(2)
            self.run()
            if (self.wumpus_world.win==True):
                break
            
        
        #self.run()
    def adder(self,count,st):
        for i in range(count):
            st = st + 1
            if(st==4):
                st = 0
        #print("ST",st)
        return st
    
    def ender(self):
        while 1:
            tempmp = self.wumpus_world.agent.currant
            self.wumpus_world.print_grid()
            if len(self.track)!=0:
                if self.priv != self.wumpus_world.agent.currant:
                    self.ch_f = True
                if self.ch_f == True:
                    self.kokko = self.track.pop()
                    print("go to back : ",self.kokko)
                    print(self.planner(self.kokko))
                    move = self.move_car(self.planner(self.kokko))
                    self.ch_f = False
                else:
                    print("go to back : ",self.kokko)
                    print(self.planner(self.kokko))
                    move = self.move_car(self.planner(self.kokko))
            else:
                break
            self.wumpus_world.print_grid()
            self.priv = tempmp
        while self.wumpus_world.agent.currant != [3,0]:
            move = self.move_car(self.planner([3,0]))
            self.wumpus_world.print_grid()
                
    
    def ender_old(self):
        while 1:
            if len(self.moves) != 0:
                ko = self.moves.pop()
                #print(self.moves)
                print("go to: ",self.back[ko])
                if self.back[ko] == "Back":
                    self.move_car('L')
                    self.move_car('L')
                    self.move_car("F")
                else:
                    self.move_car(self.back[ko])
            self.wumpus_world.print_grid()
            if self.wumpus_world.agent.currant == [3,0]:
                break
            
    def planner(self,dest):
        for i in range(4):
            #print("co coc oc")
            #print(dest)
            #print(self.wumpus_world.agent.currant)
            #print([self.wumpus_world.agent.currant[0] + self.wumpus_world.agent.move_di[i][0][0],self.wumpus_world.agent.currant[1] + self.wumpus_world.agent.move_di[i][0][1]])
            #print()
            #print(self.wumpus_world.agent.currant + self.wumpus_world.agent.move_di[i][0])
            if [self.wumpus_world.agent.currant[0] + self.wumpus_world.agent.move_di[i][0][0],self.wumpus_world.agent.currant[1] + self.wumpus_world.agent.move_di[i][0][1]] == dest:
                print(i)
                self.wumpus_world.agent.curr_di
                if i==self.wumpus_world.agent.curr_di:
                    #print('F')
                    #self.ch_f = True
                    return 'F'
                elif i == self.adder(1,self.wumpus_world.agent.curr_di):
                    return 'L'
                #print(self.adder(3,self.wumpus_world.agent.curr_di))
                elif i == self.adder(3,self.wumpus_world.agent.curr_di):
                    return 'R'
                #print("halo",self.adder(3,self.wumpus_world.agent.curr_di))
                #print(i)
        return 'R'
                
    def back_word(self):
        ko = self.moves.pop()
        if ko == "F":
            if (self.ch_f == True):
                self.move_car('L')
                self.move_car('L')
                self.move_car('F')
                self.moves.append('L')
                self.moves.append('L')
                self.moves.append('F')
                self.ch_f = False
                print("go to: Back")
            else:
                self.move_car('F')
                self.moves.append('F')
                print("go to F")
        elif ko == "L":
            self.move_car('R')
            self.moves.append('L')
            print("go to R")
        elif ko == "R":
            self.move_car('L')
            print("go to R")
            
    def back_word_2(self):
        pass
        
            
    def add_bol(self):
        #print("come in function")
        #print("previs:",self.priv)
        #print("curr:",self.wumpus_world.agent.currant)
        if (self.priv != self.wumpus_world.agent.currant):
           # print("first")
            if self.wumpus_world.agent.currant in self.track:
                jojj = len(self.track)
                for ind in range(len(self.track)):
                    if self.track[ind]==self.wumpus_world.agent.currant:
                        jojj = ind
                self.track = self.track[:jojj+1]
            #    print("second")
                
            else:
                self.track.append(self.wumpus_world.agent.currant)
             #   print("third")
            #print("chuu chale")
        print("track:",self.track)
            
        
    def temp_move(self):
        for pos in self.wumpus_world.grid[self.wumpus_world.agent.currant[0]][self.wumpus_world.agent.currant[1]].neighbor:
            #print(pos.posi)
            #print(pos.visited)
            #print(pos.ok)
            #print("pos",pos.posi)
            if (pos.ok == True and pos.visited == False):
                self.ch_f = True
                print("go to: ",pos.posi)
                #print(self.planner(pos.posi))
                self.add_bol()
                move = self.move_car(self.planner(pos.posi))
                
                
                self.moves.append(move)
                return pos.posi
        if len(self.moves) != 0:
            if self.priv != self.wumpus_world.agent.currant:
                self.ch_f = True
            if self.ch_f == True:
                self.kokko = self.track.pop()
                print("go to back : ",self.kokko)
                #print(self.planner(self.kokko))
                move = self.move_car(self.planner(self.kokko))
                self.ch_f = False
            else:
                print("go to back : ",self.kokko)
                #print(self.planner(self.kokko))
                move = self.move_car(self.planner(self.kokko))
                
                
            #self.back_word_2()
            """
            ko = self.moves.pop()
            #print(self.moves)
            print("go to: ",self.back[ko])
            if self.back[ko] == "Back":
                self.move_car('L')
                self.move_car('L')
                self.move_car("F")
            else:
                self.move_car(self.back[ko])
            """
            return 0
        print("it end")
        self.cont += 1
        if self.cont ==1:
            for ko in self.wumpus_world.grid:
                print(ko)
                ko.visited == False
            self.wumpus_world.win = True
            
        
    def move_car(self,move=None):
        
        #if move == None:
            #move = input("Move car with F L R: ")
        
        if(move=='F'):
            self.wumpus_world.agent.move_forward()
        elif(move=='L'):
            self.wumpus_world.agent.make_turn('left')
        elif(move=='R'):
            self.wumpus_world.agent.make_turn('right')
        return move
            
    def evu(self):
        
        for nigh in self.wumpus_world.rule_neighbor[self.wumpus_world.agent.currant[0]][self.wumpus_world.agent.currant[1]]:
            if ((nigh not in self.listed_do_p) and (nigh not in self.listed_done_p)):
                self.listed_do_p.append(nigh)
                #print(self.listed_do_p)
                
            if ((nigh not in self.listed_do_w) and (nigh not in self.listed_done_w)):
                self.listed_do_w.append(nigh)
                #print(self.listed_do_w)
            
        
        #temp_2,KB_cu = model_check(self.wumpus_world.kb)    
        self.predi_models, self.Total_sym = model_check(self.wumpus_world.kb,self.Total_sym,self.predi_models)
        KB_cu = self.predi_models.copy()
        temp = self.listed_do_p.copy()        
        for nigh in temp:                       #print(self.wumpus_world.di) #if model_check(self.wumpus_world.kb, eval(f'self.wumpus_world.di[f"P{nigh}"]'))==True:
            
            if check_res(KB_cu,'P'+nigh,True):  
                print(f"pit at {nigh}")
                #print()
                self.wumpus_world.kb.add(eval(self.wumpus_world.join_sym("",[f"self.wumpus_world.di['P{nigh}']"])))
                self.listed_do_p.remove(nigh)
                self.listed_done_p.append(nigh)
                self.wumpus_world.grid[int(nigh[0]),int(nigh[1])].pit=True

                
            #elif model_check(self.wumpus_world.kb, eval(f'Not(self.wumpus_world.di[f"P{nigh}"])'))==True:
            elif check_res(KB_cu,'P'+nigh,False):    
                print(f"No pit at {nigh}")
                #print(eval(self.wumpus_world.join_sym("Not",[f"self.wumpus_world.di[P{nigh}]"])))
                self.wumpus_world.kb.add(eval(self.wumpus_world.join_sym("Not",[f"self.wumpus_world.di['P{nigh}']"])))
                #print(self.wumpus_world.kb)
                self.listed_do_p.remove(nigh)
                self.listed_done_p.append(nigh)
                self.wumpus_world.grid[int(nigh[0]),int(nigh[1])].pit=False
                if ((nigh in self.listed_done_w) and (self.wumpus_world.grid[int(nigh[0]),int(nigh[1])].wumpus==False)):
                    self.wumpus_world.grid[int(nigh[0]),int(nigh[1])].ok=True
                    self.ok.append(nigh)
           
        temp = self.listed_do_w.copy() 
        for nigh in temp:
                
            if check_res(KB_cu,'W'+nigh,True):  
                print(f"wumpus at {nigh}")
                #print()
                
                self.wumpus_world.kb.add(eval(self.wumpus_world.join_sym("",[f"self.wumpus_world.di['W{nigh}']"])))
                self.listed_do_w.remove(nigh)
                self.listed_done_w.append(nigh)
                self.wumpus_world.grid[int(nigh[0]),int(nigh[1])].wumpus=True

            #elif model_check(self.wumpus_world.kb, eval(f'Not(self.wumpus_world.di[f"P{nigh}"])'))==True:
            elif check_res(KB_cu,'W'+nigh,False):    
                print(f"No wumpus at {nigh}")
                #print(eval(self.wumpus_world.join_sym("Not",[f"self.wumpus_world.di[P{nigh}]"])))
                self.wumpus_world.kb.add(eval(self.wumpus_world.join_sym("Not",[f"self.wumpus_world.di['W{nigh}']"])))
                #print(self.wumpus_world.kb)
                self.listed_do_w.remove(nigh)
                self.listed_done_w.append(nigh)
                self.wumpus_world.grid[int(nigh[0]),int(nigh[1])].wumpus=False
                if ((nigh in self.listed_done_p) and (self.wumpus_world.grid[int(nigh[0]),int(nigh[1])].pit==False)):
                    self.wumpus_world.grid[int(nigh[0]),int(nigh[1])].ok=True
                    self.ok.append(nigh)
                    
        
    def run(self):
        print(self.track)
        #print([self.wumpus_world.agent.currant[0],self.wumpus_world.agent.currant[1]])
        
        print("come")
        print(self.wumpus_world.agent.currant)
        if(self.wumpus_world.agent.visited[self.wumpus_world.agent.currant[0],self.wumpus_world.agent.currant[1]]==0):
            self.wumpus_world.grid[self.wumpus_world.agent.currant[0],self.wumpus_world.agent.currant[1]].visited = 1
            #print("ki")
            if (self.wumpus_world.get_array()==[1,0,0,0,0]):
                self.ender()
                self.wumpus_world.win=True
                return 0
            self.wumpus_world.rule_add()
            self.evu()
            print("ok its",self.ok)
            temp = self.wumpus_world.agent.currant.copy()
            self.temp_move()
            self.priv = temp
            
            
        else:
            print("ok its",self.ok)
            temp = self.wumpus_world.agent.currant.copy()
            #print("previous:",self.wumpus_world.agent.currant)
            self.temp_move()
            self.priv = temp
            #print("gone to ",self.wumpus_world.agent.currant)
            #self.wumpus_world.print_grid()
        self.wumpus_world.print_grid()

kkkk = Game()
kkkk.start()

ModuleNotFoundError: No module named 'RPi'

In [ ]:
kkkk = Game()
kkkk.start()

In [ ]:
if 7>6:
    print(8)
elif 9>8:
    print(7)
if 5>6:
    print(8)
elif 9>8:
    print(7)

In [ ]:
back = {"F":"Back",
                    "L":"R",
                    "R":"L"}

In [ ]:
back['F']

In [ ]:
[0,1] + [1,0]

In [ ]:
[0,1][0]

In [ ]:
[0,1] == [1,0]

In [10]:
oo = [1,7,8,2,9]

In [11]:
if 6 in oo:
    j = len(oo)
    for i in range(len(oo)):
        if oo[i] == 6:
            j = i
    oo = oo[:j+1]
else:
    oo.append(6)
print(j)
print(oo)

2
[1, 7, 8, 2, 9, 6]


In [9]:
oo[:3]

[1, 7, 6]